In [0]:
#fill in directory and key
spark.sparkContext.hadoopConfiguration.set()

In [0]:
from pyspark.sql import functions as F
from functools import reduce
from pyspark.sql.types import StringType
import re

In [0]:
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

#Create dataframes from csv files

Create dataframes from all the useful tables, with only useful columns, in English.

List of created dataframes:
- dfFlockPresentation
- dfFarmer
- dfVisitReason
- dfFlockPresentationPresentation
- dfFlockPresentationDiagnosis
- dfFlockPresentationEtiology
- dfFlockPresentationPen
- dfSubscription (selection on antibiotics performed later on)
- dfPresentation
- dfPresentationCategory
- dfEtiology
- dfDiagnosis
- dfDiagnosisGroup
- dfDrug  (dfAntibiotic is only antibiotics)
- dfFlock
- dfPen
- dfSupply
- dfPackage
- dfCompanyType: instead of merging this table with the main table, also possible to replace CompanyType '13' with 'OnFarmHatching' and CompanyType '50' with 'Broiler' (the only ones in the dataset)
- dfVetPractice: for veterinary practice ID

In [0]:
df = spark. \
  read. \
  option("header", "true"). \
  option("delimiter", ","). \
  csv("//vmp_koppeltype.csv")

In [0]:
dfHouder = spark. \
  read. \
  option("header", "true"). \
  option("delimiter", ","). \
  csv("//vmp_houder.csv")

dfFarmer = dfHouder \
    .withColumnRenamed('HDRID','FarmerIdentification') \
    .withColumnRenamed('KIPNUMMER','PoultryFarmIdentification') \
    .withColumnRenamed('UBN','FarmIdentification') \
    .withColumnRenamed('POSTCODE', 'PostalCode') \
    .withColumnRenamed('BEDRIJFSOORT','CompanyType') \
    .drop('WOONPLAATS', 'ADRES', 'KIPBEGINDATUM', 'NAAM', 'DATUM_LAMU', 'USER_LAMU', 'KIPEINDDATUM', 'PMPKUBID', 'STRAAT', 'HUISNR', 'HUISNRTOEV', 'REGISTRATIENUMMER', 'POSTPLAATS', 'POSTADRES', 'POSTPOSTCODE', 'POSTSTRAAT', 'POSTHUISNR', 'POSTHUISNRTOEV', 'SRTCODE', 'PMPKIPID', 'POSTCODE')

#change Company Type values with full names:
dfFarmer = dfFarmer.withColumn('CompanyType', F.regexp_replace('CompanyType', '50', 'BroilerFarm')) \
    .withColumn('CompanyType', F.regexp_replace('CompanyType', '13', 'Patio'))

In [0]:
#this is the main table

dfKoppelbeeld = spark. \
  read. \
  option("header", "true"). \
  option("delimiter", ","). \
  option("multiline","true"). \
  csv("//vmp_koppelbeeld.csv")

dfFlockPresentation1 = dfKoppelbeeld \
    .withColumnRenamed('KBLID','FlockPresentationIdentification') \
    .withColumnRenamed('HDRID','FarmerIdentification') \
    .withColumnRenamed('CONTACTDATUM','ContactDate') \
    .withColumnRenamed('RELATIENRDAP','VeterinaryPracticeNumber') \
    .withColumnRenamed('OPMERKING','Note') \
    .withColumnRenamed('RELATIENRDA','VeterinarianNumber') \
    .drop('DATUM_LAMU', 'RELNR_LAMU', 'IENR_OPMERKING', 'USER_LAMU', 'SECTIEBEVINDINGEN', 'RBZID')

# Necropsy findings and visit reason dropped because mostly NA.

# Change data type
dfFlockPresentation = dfFlockPresentation1 \
    .withColumn('ContactDate',
                F.to_timestamp(dfFlockPresentation1['ContactDate'], "dd-MM-yy"))

In [0]:
dfFlockPresentationFiltered = dfFlockPresentation \
    .filter(F.col("FlockPresentationIdentification").rlike("^[0-9]+$"))

In [0]:
dfKoppelBeeldBeeld = spark. \
  read. \
  option("header", "true"). \
  option("delimiter", ","). \
  csv("//vmp_koppelbeeldbeeld.csv")

dfFlockPresentationPresentation = dfKoppelBeeldBeeld.withColumnRenamed('KBBID','FlockPresentationPresentationIdentification') \
    .withColumnRenamed('BLDID','PresentationIdentification') \
    .withColumnRenamed('KBLID','FlockPresentationIdentification') \
    .drop('DATUM_LAMU', 'RELNR_LAMU', 'USER_LAMU')

In [0]:
dfKoppelBeeldDiagnose = spark. \
  read. \
  option("header", "true"). \
  option("delimiter", ","). \
  csv("//vmp_koppelbeelddiagnose.csv")

dfFlockPresentationDiagnosis = dfKoppelBeeldDiagnose.withColumnRenamed('KBDID','FlockPresentationDiagnosisIdentification') \
    .withColumnRenamed('DGNID','DiagnosisIdentification') \
    .withColumnRenamed('KBLID','FlockPresentationIdentification') \
    .drop('DATUM_LAMU', 'RELNR_LAMU', 'USER_LAMU')

In [0]:
dfKoppelBeeldEtiologie = spark. \
  read. \
  option("header", "true"). \
  option("delimiter", ","). \
  csv("//vmp_koppelbeeldetiologie.csv")

dfFlockPresentationEtiology = dfKoppelBeeldEtiologie.withColumnRenamed('KBEID','FlockPresentationEtiologyIdentification') \
    .withColumnRenamed('ETHID','EtiologyIdentification') \
    .withColumnRenamed('KBLID','FlockPresentationIdentification') \
    .drop('DATUM_LAMU', 'RELNR_LAMU', 'USER_LAMU')

In [0]:
dfKoppelBeeldHok = spark. \
  read. \
  option("header", "true"). \
  option("delimiter", ","). \
  csv("//vmp_koppelbeeldhok.csv")

In [0]:
dfFlockPresentationPen = dfKoppelBeeldHok \
    .withColumnRenamed('KBHID','FlockPresentationPenIdentification') \
    .withColumnRenamed('HOKID','PenIdentification') \
    .withColumnRenamed('KBLID','FlockPresentationIdentification') \
    .drop('DATUM_LAMU', 'RELNR_LAMU', 'USER_LAMU', 'HANEN', 'HENNEN', 'ONGESEXT', 'AANTAL_HANEN', 'AANTAL_HENNEN', 'AANTAL_ONGESEXT')

# one FlockPresentationIdentification can belong to multiple PenIdentifications

In [0]:
dfVoorschrift = spark. \
  read. \
  option("header", "true"). \
  option("delimiter", ","). \
  csv("//vmp_koppelbeeldvoorschrift.csv")

dfSubscription = dfVoorschrift.withColumnRenamed('KBVID','SubscriptionIdentification') \
    .withColumnRenamed('BEGINDATUM','TreatmentStartDate') \
    .withColumnRenamed('GNMID','DrugIdentification') \
    .withColumnRenamed('KUURDUURDAGEN','TreatmentDays') \
    .withColumnRenamed('DOSERINGHOEVEELHEID','TreatmentDosage') \
    .withColumnRenamed('KBLID','FlockPresentationIdentification') \
    .withColumnRenamed('DOSERINGEENHEID','TreatmentDosageUnit') \
    .drop('DATUM_LAMU', 'RELNR_LAMU', 'USER_LAMU', 'ISCONFORMBIJSLUITER', 'MELDDATUM', 'OPMERKING')

#change data type:
dfSubscription = dfSubscription \
    .withColumn('TreatmentDays',
                dfSubscription['TreatmentDays'] \
                .cast("float")) \
    .withColumn('TreatmentDosage',
                dfSubscription['TreatmentDosage'] \
                .cast("integer")) \
    .withColumn('TreatmentStartDate',
                F.to_timestamp(dfSubscription['TreatmentStartDate'], "yyyy-MM-dd")) \
    .filter(F.col('TreatmentStartDate') >= "2013-01-01 00:00:00") \
    .filter(F.col('TreatmentStartDate') <= "2021-12-31 00:00:00")

#dfSubscription.printSchema()

In [0]:
dfLevering = spark. \
  read. \
  option("header", "true"). \
  option("delimiter", ","). \
  option("multiline","true"). \
  csv("//vmp_koppelbeeldlevering.csv")

dfSupply = dfLevering \
    .withColumnRenamed('KBLID','FlockPresentationIdentification') \
    .withColumnRenamed('VRPID','PackageIdentification') \
    .withColumnRenamed('DATUM','TreatmentSupplyDate') \
    .withColumnRenamed('AANTAL','NumberSupplied') \
    .drop('LVRID', 'DATUM_LAMU', 'RELNR_LAMU', 'MELDDATUM', 'USER_LAMU', 'OPMERKING', 'KBVID')

#change data type:
dfSupply = dfSupply \
    .withColumn('TreatmentSupplyDate',
                F.to_timestamp(dfSupply['TreatmentSupplyDate'], "yyyy-MM-dd")) \
    .withColumn('NumberSupplied',
                dfSupply['NumberSupplied'] \
                .cast("integer"))

In [0]:
dfVerpakking = spark. \
  read. \
  option("header", "true"). \
  option("delimiter", ","). \
  option("multiline","true"). \
  csv("//vmp_verpakking.csv")

dfPackage = dfVerpakking \
    .withColumnRenamed('VRPID','PackageIdentification') \
    .withColumnRenamed('PRODUCTNAAM','ProductName') \
    .withColumnRenamed('GNMID','DrugIdentification') \
    .withColumnRenamed('BEH_KG_KIP','KilogramsToTreat') \
    .drop('EANCODE', 'VERSIENUMMER', 'VERPAKKINGNAAM', 'VERPAKKINGAANTAL', 'VERPAKKINGEENHEID', 'INGANGSDATUM', 'AANMAAKDATUM',
         'DATUM_LAMU','BRON','REGNL','MASTEREANCODE','BEH_KG_KALKOEN','BEH_KG_EEND','USER_LAMU','RELNR_LAMU','LAATSTEWIJZIGING')

# note: 'VERPAKKINGAANTAL' is the weight of the package in mg, g, or kg (in case of antibiotics).
# 'KilogramsToTreat' is the kilograms of chicken treatable with the package.

#change data type:
dfPackage = dfPackage \
    .withColumn('KilogramsToTreat',
                dfPackage['KilogramsToTreat'] \
                .cast("integer"))

In [0]:
dfBeeld = spark. \
  read. \
  option("header", "true"). \
  option("delimiter", ","). \
  csv("//vmp_beeld.csv")

dfPresentation = dfBeeld.withColumnRenamed('BLDID','PresentationIdentification') \
    .withColumnRenamed('OMSCHRIJVING','Presentation') \
    .withColumnRenamed('BLCID','PresentationCategoryIdentification') \
    .drop('EWS','VOLGORDE')

In [0]:
dfBeeldCategorie = spark. \
  read. \
  option("header", "true"). \
  option("delimiter", ","). \
  csv("//vmp_beeldcategorie.csv")

dfPresentationCategory = dfBeeldCategorie.withColumnRenamed('BLCID','PresentationCategoryIdentification') \
    .withColumnRenamed('OMSCHRIJVING','PresentationCategory') \
    .drop('VOLGORDE')

In [0]:
dfEtiologie = spark. \
  read. \
  option("header", "true"). \
  option("delimiter", ","). \
  csv("//vmp_etiologie.csv")

dfEtiology = dfEtiologie.withColumnRenamed('ETHID','EtiologyIdentification') \
    .withColumnRenamed('CODE','EtiologyCode') \
    .withColumnRenamed('OMSCHRIJVING','EtiologyDescription') \
    .withColumnRenamed('VIRAAL','EtiologyViral') \
    .withColumnRenamed('BACTERIE','EtiologyBacterial') \
    .withColumnRenamed('PARASIET','EtiologyParasitic') \
    .withColumnRenamed('SCHIMMEL','EtiologyFungal') \
    .withColumnRenamed('NIET_INFECTIEUS','EtiologyNonInfectious')

In [0]:
dfDiagnose = spark. \
  read. \
  option("header", "true"). \
  option("delimiter", ","). \
  csv("//vmp_diagnose.csv")

dfDiagnosis = dfDiagnose.withColumnRenamed('DGNID','DiagnosisIdentification') \
    .withColumnRenamed('CODE','DiagnosisCode') \
    .withColumnRenamed('OMSCHRIJVING','DiagnosisDescription') \
    .withColumnRenamed('DGRID','DiagnosisGroupIdentification') \
    .withColumnRenamed('VIRAAL','DiagnosisViral') \
    .withColumnRenamed('BACTERIE','DiagnosisBacterial') \
    .withColumnRenamed('PARASIET','DiagnosisParasitic') \
    .withColumnRenamed('SCHIMMEL','DiagnosisFungal') \
    .withColumnRenamed('NIET_INFECTIEUS','DiagnosisNonInfectious')

In [0]:
dfDiagnoseGroep = spark. \
  read. \
  option("header", "true"). \
  option("delimiter", ","). \
  csv("//vmp_diagnosegroep.csv")

dfDiagnosisGroup = dfDiagnoseGroep.withColumnRenamed('DGRID','DiagnosisGroupIdentification') \
    .withColumnRenamed('Groep','DiagnosisGroup')

In [0]:
#NB: dfAntibiotic is antibiotics only

dfGeneesmiddel = spark. \
  read. \
  option("header", "true"). \
  option("delimiter", ","). \
  csv("//vmp_geneesmiddel.csv")

dfDrug = dfGeneesmiddel.withColumnRenamed('GNMID','DrugIdentification') \
    .withColumnRenamed('OMSCHRIJVING','DrugDescription') \
    .withColumnRenamed('WERKZAMESTOF','DrugActiveSubstance') \
    .withColumnRenamed('ISANTIBIOTICA','IsAntibiotics') \
    .withColumnRenamed('KEUZE_ANTIBIOTICA','ChoiceAntibiotics') \
    .drop('REGNL', 'FIRMA', 'WACHTTIJDEI', 'BRON', 'ISVACCIN', 'DATUM_INS', 'DATUM_LAMU', 'ISACTIEF', 'ISCASCADE', 'ISCASCADE_KALKOEN', 'WACHTTIJDVLEES', 'TH_GROEP_SDA', 'IN_FORMULARIUM_KALKOEN', 'USER_LAMU', 'RELNR_LAMU')
 
#IsAntibiotics is Yes (J) or No (N)
#Selection on antibiotics:
dfAntibiotic = dfDrug.filter(dfDrug['IsAntibiotics'] == 'J')
dfDrugNoAntibiotic = dfDrug.filter(dfDrug['IsAntibiotics'] == 'N')

In [0]:
def pen_correction(text):
    dict = {
    " " : "",
    "-" : ""} 
    
    #if value is Null, do nothing
    if text is not None:
        # Create a regular expression  from the dictionary keys
        regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))
        # For each match, look-up corresponding value in dictionary
        text = regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text)
    return text

pen_correction_udf = udf(pen_correction, StringType())

In [0]:
# Note: both PenIdentification and ResidenceIdentification seem to be in the table 'Flock' as well. But this table provides a link with ResidenceNumber, which might be the key for linkage with GD data. (?)
# ALSO: ResidenceIdentification is not complete in 'Flock'.

dfHok = spark. \
  read. \
  option("header", "true"). \
  option("delimiter", ","). \
  csv("//vmp_hok.csv")

dfPen = dfHok.withColumnRenamed('HOKID','PenIdentification') \
    .withColumnRenamed('VERBLIJFPLAATSNUMMER','PenNumber') \
    .withColumn('PenNumber',
                F.explode(
                    F.split(
                        F.lower(pen_correction_udf(
                            F.col('PenNumber'))),
                        '\+|,'))) \
    .drop('DATUM_LAMU', 'USER_LAMU', 'PMPHOKID', 'INGANGSDATUM', 'VERVALDATUM', 'HDRID', 'KIPVERBLIJFPLAATSID', 'AANDUIDING')

# not splitting on . and - is on purpose
# after exploding: 17 additional rows (0.5%)

In [0]:
dfDAP = spark. \
  read. \
  option("header", "true"). \
  option("delimiter", ","). \
  csv("//vmp_dap.csv")

dfVeterinaryPractice = dfDAP \
    .withColumnRenamed('RELATIE_NR','VeterinaryPracticeNumber') \
    .withColumnRenamed('DAPNR','VeterinaryPracticeIdentification') \
    .drop('NAAM', 'WOONPLAATS', 'ADRES', 'TELEFOON')

# Merge tables

As last step, merge all tables with the main table FlockPresentation.

In [0]:
# Merge dfPresentationCategory with dfPresentation through key 'PresentationCategoryIdentification' (BLCID)
dfPresentation2 = dfPresentation.join(dfPresentationCategory, on=['PresentationCategoryIdentification'], how='left_outer') \
    .drop('PresentationCategoryIdentification')

# Merge dfPresentation2 with dfFlockPresentationPresentation through key 'PresentationIdentification' (BLDID)
dfFlockPresentationPresentation2 = dfFlockPresentationPresentation.join(dfPresentation2, on=['PresentationIdentification'], how='left_outer') \
    .drop('PresentationIdentification','FlockPresentationPresentationIdentification')

#Convert to one entry per FlockPresentationIdentification: concacenate strings in 'Presentation' and 'PresentationCategory'
dfFlockPresentationPresentation3 = dfFlockPresentationPresentation2.groupby('FlockPresentationIdentification') \
    .agg(
        F.collect_list('Presentation').alias("Presentation"),
        F.collect_list('PresentationCategory').alias("PresentationCategory"))\
    .withColumn("Presentation", F.concat_ws(", ", "Presentation")) \
    .withColumn("PresentationCategory", F.concat_ws(", ", "PresentationCategory"))

In [0]:
# Merge dfDiagnosisGroup with dfDiagnosis through key 'DiagnosisGroupIdentification' (DGRID)
dfDiagnosis2 = dfDiagnosis.join(dfDiagnosisGroup, on=['DiagnosisGroupIdentification'], how='left_outer') \
    .drop('DiagnosisGroupIdentification')

# Merge dfFlockPresentationDiagnosis with dfDiagnosis2 through key 'DiagnosisIdentification' (DGNID)
dfFlockPresentationDiagnosis2 = dfFlockPresentationDiagnosis.join(dfDiagnosis2, on=['DiagnosisIdentification'], how='left_outer') \
    .drop('FlockPresentationDiagnosisIdentification','DiagnosisIdentification')

#Convert to one entry per FlockPresentationIdentification: concacenate strings in all other columns
dfFlockPresentationDiagnosis3 = dfFlockPresentationDiagnosis2.groupby('FlockPresentationIdentification') \
    .agg(
        F.collect_list('DiagnosisCode').alias("DiagnosisCode"),
        F.collect_list('DiagnosisDescription').alias("DiagnosisDescription"),
        F.collect_list('DiagnosisViral').alias("DiagnosisViral"),
        F.collect_list('DiagnosisBacterial').alias("DiagnosisBacterial"),
        F.collect_list('DiagnosisParasitic').alias("DiagnosisParasitic"),
        F.collect_list('DiagnosisFungal').alias("DiagnosisFungal"),
        F.collect_list('DiagnosisNonInfectious').alias("DiagnosisNonInfectious"),
        F.collect_list('DiagnosisGroup').alias("DiagnosisGroup"))\
    .withColumn("DiagnosisCode", F.concat_ws(", ", "DiagnosisCode")) \
    .withColumn("DiagnosisDescription", F.concat_ws(", ", "DiagnosisDescription")) \
    .withColumn("DiagnosisViral", F.concat_ws(", ", "DiagnosisViral")) \
    .withColumn("DiagnosisBacterial", F.concat_ws(", ", "DiagnosisBacterial")) \
    .withColumn("DiagnosisParasitic", F.concat_ws(", ", "DiagnosisParasitic")) \
    .withColumn("DiagnosisFungal", F.concat_ws(", ", "DiagnosisFungal")) \
    .withColumn("DiagnosisNonInfectious", F.concat_ws(", ", "DiagnosisNonInfectious")) \
    .withColumn("DiagnosisGroup", F.concat_ws(", ", "DiagnosisGroup"))
    

In [0]:
# Merge dfFlockPresentationEtiology with dfEtiology through key 'EtiologyIdentification' (ETHID)
dfFlockPresentationEtiology2 = dfFlockPresentationEtiology.join(dfEtiology, on=['EtiologyIdentification'], how='left_outer') \
    .drop('FlockPresentationEtiologyIdentification','EtiologyIdentification')

# The same FlockPresentationIdentification can have duplicate EtiologyCodes. So filter on distinct().
# Also convert to one entry per FlockPresentationIdentification: concacenate strings in all other columns.

dfFlockPresentationEtiology3 = dfFlockPresentationEtiology2 \
    .distinct() \
    .groupby('FlockPresentationIdentification') \
    .agg(
        F.collect_list('EtiologyCode').alias("EtiologyCode"),
        F.collect_list('EtiologyDescription').alias("EtiologyDescription"),
        F.collect_list('EtiologyViral').alias("EtiologyViral"),
        F.collect_list('EtiologyBacterial').alias("EtiologyBacterial"),
        F.collect_list('EtiologyParasitic').alias("EtiologyParasitic"),
        F.collect_list('EtiologyFungal').alias("EtiologyFungal"),
        F.collect_list('EtiologyNonInfectious').alias("EtiologyNonInfectious")) \
    .withColumn("EtiologyCode", F.concat_ws(", ", "EtiologyCode")) \
    .withColumn("EtiologyDescription", F.concat_ws(", ", "EtiologyDescription")) \
    .withColumn("EtiologyViral", F.concat_ws(", ", "EtiologyViral")) \
    .withColumn("EtiologyBacterial", F.concat_ws(", ", "EtiologyBacterial")) \
    .withColumn("EtiologyParasitic", F.concat_ws(", ", "EtiologyParasitic")) \
    .withColumn("EtiologyFungal", F.concat_ws(", ", "EtiologyFungal")) \
    .withColumn("EtiologyNonInfectious", F.concat_ws(", ", "EtiologyNonInfectious"))

In [0]:
# dfSupply contains 100% PackageIdentification, dfPackage contains 100% DrugIdentification.

dfSupply2 = dfSupply \
    .join(dfPackage, on=['PackageIdentification'], how='left_outer') \
    .join(dfDrug, on=['DrugIdentification'], how='left_outer') \
    .drop('DrugIdentification','PackageIdentification')

In [0]:
# Merge dfFlockPresentationPen with dfPen through key 'PenIdentification' (HOKID)
dfFlockPresentationPen2 = dfFlockPresentationPen.join(dfPen, on=['PenIdentification'], how='left_outer') \
    .drop('FlockPresentationPenIdentification','PenIdentification')

# One FlockPresentationIdentification can contain multiple pens. This is good, because we want to seperate on pen.
# The total dataset (dfFlockPresentationFull) should be longer than dfFlockPresentation.

In [0]:
dfNumberOfPensPerFlockPresentation = dfFlockPresentationFiltered \
    .join(dfFlockPresentationPen2, on=['FlockPresentationIdentification'], how='left_outer') \
    .groupBy('FlockPresentationIdentification').count() \
    .withColumn('count', F.col('count').cast("integer"))

In [0]:
# dfFlockPresentationFiltered because then all FlockPresentationIdentifications are correct numbers.
# Merge dfFlockPresentation with dfFlockPresentationPresentation2 through key 'FlockPresentationIdentification' (KBLID)
# Merge dfFlockPresentation with dfFlockPresentationDiagnosis2 through key 'FlockPresentationIdentification' (KBLID)
# Merge dfFlockPresentation with dfFlockPresentationEtiology2 through key 'FlockPresentationIdentification' (KBLID)
# Merge dfFlockPresentation with dfSupply2 through key 'FlockPresentationIdentification' (KBLID)
# Merge dfFlockPresentation with dfFlockPresentationPen2 through key 'FlockPresentationIdentification' (KBLID)
# Merge dfFlockPresentation with dfFarmer through key 'FarmerIdentification'
# Merge dfVeterinaryPractice. Rename VeterinaryPractiIdentification as VeterinaryPracticeNumber, so that nothing has to be changed in later steps. VeterinaryPracticeNumber is now the same as in NDV titer 

# And exclude contact dates 2010-2012

# merge with dfNumberOfPensPerFlockPresentation to get a column with the number of pens belonging to that FlockPresentationID (useful for calculating kg antibiotics)

dfFlockPresentationFull = dfFlockPresentationFiltered \
    .join(dfFlockPresentationPen2, on=['FlockPresentationIdentification'], how='left_outer') \
    .join(dfNumberOfPensPerFlockPresentation, on=['FlockPresentationIdentification'], how='left_outer') \
    .join(dfFlockPresentationPresentation3, on=['FlockPresentationIdentification'], how='left_outer') \
    .join(dfFlockPresentationDiagnosis3, on=['FlockPresentationIdentification'], how='left_outer') \
    .join(dfFlockPresentationEtiology3, on=['FlockPresentationIdentification'], how='left_outer') \
    .join(dfSupply2, on=['FlockPresentationIdentification'], how='left_outer') \
    .join(dfSubscription, on=["FlockPresentationIdentification"], how = 'left_outer') \
    .join(dfFarmer, on=['FarmerIdentification'], how='left_outer') \
    .join(dfVeterinaryPractice, on=['VeterinaryPracticeNumber'], how = 'left_outer') \
    .drop('FarmerIdentification', 'VeterinaryPracticeNumber') \
    .withColumnRenamed('VeterinaryPracticeIdentification', 'VetId')
   
# Note: this dataset contains A LOT of entries without supply of medicines.
# They are filtered in dfAntibiotics.

In [0]:
dfFlock = dfKoppel.withColumnRenamed('KPLID','FlockIdentification') \
    .withColumnRenamed('GEBOORTEDATUM','HatchDate') \
    .withColumnRenamed('AFVOERDATUM','RemovalDate') \
    .withColumnRenamed('AANTALOPGEZET','NumberPlaced') \
    .withColumnRenamed('RASCODE','BreedCode') \
    .withColumnRenamed('HDRID','FarmerIdentification') \
    .withColumnRenamed('HOKID','PenIdentification') \
    .withColumnRenamed('RASOMSCHRIJVING','BreedDescription') \
    .withColumnRenamed('AANTALLEVENSDAGEN','NumberOfLifeDays') \
    .withColumnRenamed('OPZETDATUM','PlacementDate') \
    .withColumnRenamed('AANTALOPGEZET_HANEN','NumberOfRoostersPlaced') \
    .withColumnRenamed('AFVOERDATUMGEPLAND','PlannedRemovalDate') \
    .withColumnRenamed('GROEICURVESOORT','GrowthCurve') \
    .withColumnRenamed('PMPKOPPELID', 'MonitoringIdentification') \
    .join(dfFarmer, on=['FarmerIdentification'], how='left_outer') \
    .join(dfPen, on=['PenIdentification'], how='left_outer') \
    .drop('KIPKOPPELID', 'KIPVERBLIJFPLAATSID', 'DATUM_LAMU', 'USER_LAMU', 'MonitoringIdentification', 'PVTID',
          'NumberOfRoostersPlaced', 'AANTALOPGEZET_HENNEN', 'AANTALOPGEZET_ONGESEXT', 'LIJNINFO', 'PostalCode', 'CompanyType')

---------------------------------------------------------------------------
NameError Traceback (most recent call last)
File :1
----> 1 dfFlock = dfKoppel.withColumnRenamed('KPLID','FlockIdentification') \
 2 .withColumnRenamed('GEBOORTEDATUM','HatchDate') \
 3 .withColumnRenamed('AFVOERDATUM','RemovalDate') \
 4 .withColumnRenamed('AANTALOPGEZET','NumberPlaced') \
 5 .withColumnRenamed('RASCODE','BreedCode') \
 6 .withColumnRenamed('HDRID','FarmerIdentification') \
 7 .withColumnRenamed('HOKID','PenIdentification') \
 8 .withColumnRenamed('RASOMSCHRIJVING','BreedDescription') \
 9 .withColumnRenamed('AANTALLEVENSDAGEN','NumberOfLifeDays') \
 10 .withColumnRenamed('OPZETDATUM','PlacementDate') \
 11 .withColumnRenamed('AANTALOPGEZET_HANEN','NumberOfRoostersPlaced') \
 12 .withColumnRenamed('AFVOERDATUMGEPLAND','PlannedRemovalDate') \
 13 .withColumnRenamed('GROEICURVESOORT','GrowthCurve') \
 14 .withColumnRenamed('PMPKOPPELID', 'MonitoringIdentification') \
 15 .join(dfFarmer, on=['FarmerIdentification'], how='left_outer') \
 16 .join(dfPen, on=['PenIdentification'], how='left_outer') \
 17 .drop('KIPKOPPELID', 'KIPVERBLIJFPLAATSID', 'DATUM_LAMU', 'USER_LAMU', 'MonitoringIdentification', 'PVTID',
 18 'NumberOfRoostersPlaced', 'AANTALOPGEZET_HENNEN', 'AANTALOPGEZET_ONGESEXT', 'LIJNINFO', 'PostalCode', 'CompanyType')

NameError: name 'dfKoppel' is not defined

In [0]:
dfFlock.write \
    .format("parquet") \
    .mode("overwrite") \
    .save("//dfFlock.parquet")

# Create events: antibiotics

First, make dataset with only antibiotics.


In [0]:
dfAntibiotics = dfFlockPresentationFull \
    .drop('DiagnosisDescription','DiagnosisViral','DiagnosisBacterial','DiagnosisParasitic','DiagnosisFungal','DiagnosisNonInfectious',
        'EtiologyDescription','EtiologyViral','EtiologyBacterial','EtiologyParasitic','EtiologyFungal','EtiologyNonInfectious') \
    .filter(F.col('IsAntibiotics')=="J") \
    .filter(F.col('TreatmentSupplyDate') >= "2013-01-01 00:00:00") \
    .filter(F.col('TreatmentSupplyDate') <= "2021-12-31 00:00:00") \
    .withColumn('KilogramsTreated', (F.col('NumberSupplied')*F.col('KilogramsToTreat'))/F.col('count')) \
    .drop('KilogramsToTreat', 'count')

In [0]:
dfAntibiotics.write \
    .format("parquet") \
    .mode("overwrite") \
    .save("//dfAntibiotics.parquet")
# first version: 8/7/22
# edits:
# - 19/07/22 filtered dfFlockPresentation on date (>= 2013),
#     grouped etiology codes together in one row per flockPresentationIdentification,
#     added distinct() in dfSubscription and in dfAntibiotics (further removes 49 duplicates)
# - 22/07/22 added dfSupply for the TreatmentSupplyDate
# - 26/07/22 added dfSupply properly, removed VisitReason and Subscription, removed columns about Pen, ResidenceNumber -> PenNumber
# - 23/09/22 removed filter contact date (because you don't want to miss any antibiotics), added filter supply date <= 2021 (because otherwise last flocks have antibiotics from 2022 flocks in them)
# - 14/11/22 added veterinary practice number (the good one)
# - 18/11/22 changed "VeterinaryPracticeNumber" to "VetId"
# - 25/11/22 added pen number correction
# - 19/12/22 whole postal code
# - 06/05/23 corrected KilogramsTreated: now multiplied by number of packages supplied
# - 08/05/23 corrected KilogramsTreated again: now divided by number of pens in one flockpresentation
# - 24/10/23 added dfSubscription with treatment days for DDD calculation

In [0]:
dfAntibiotics = spark.read.parquet("//dfAntibiotics.parquet")

In [0]:
# fill in for the specific dataset:
Type = "Antibiotics"
EventDate = "TreatmentSupplyDate"
Pen = "PenNumber"
df = dfAntibiotics

RightColumns = ["FarmIdentification",
                Pen,
                EventDate,
                "EventType"]
OtherColumns = sorted(list(set(df.columns) - set(RightColumns)))

# change df name:
# create column EventDate and Type, put all other columns in metadata column, drop all other columns
dfAntibioticEventsStruct = df \
    .withColumnRenamed(EventDate, "EventDate") \
    .withColumnRenamed(Pen, "Pen") \
    .withColumn("EventType", F.lit(Type)) \
    .withColumn("MetaData", F.struct(*OtherColumns)) \
    .select("FarmIdentification", "Pen", "EventDate", "EventType", "MetaData")

In [0]:
dfAntibioticEventsStruct.write \
    .format("parquet") \
    .mode("overwrite") \
    .save("//dfAntibioticEventsStruct.parquet")
# This is with StructType instead of MapType
# first version: 29/06/22    (with the dates as DateType)
# edits:
#    - 07/07/22 Included Pen column
#    - 19/7/22 Used the new dfAntibiotics (see changes there)
#    - 22/7/22 Used new dfAntibiotics with TreatmentSupplyDate, EventType = "Antibiotics", EventDate = "TreatmentSupplyDate"
#    - 26/7/22 Used new and improved dfAntibiotics with TreatmentSupplyDate, Pen = "PenNumber". Changed the columns in MetaData
#    - 23/9/22 Changed filters (see dfAntibiotics)
#    - 14/11/22 Added veterinary practice number (the right one)
#    - 06/05/23 Corrected KilogramsTreated
#    - 08/05/23 corrected KilogramsTreated (see dfAntibiotics)
#    - 24/10/23 added dfSubscription with treatment days for DDD calculation

# Create events: other registrations

In [0]:
dfNotAntibiotics = dfFlockPresentationFull \
    .drop('DiagnosisDescription','DiagnosisViral','DiagnosisBacterial','DiagnosisParasitic','DiagnosisFungal','DiagnosisNonInfectious',
        'EtiologyDescription','EtiologyViral','EtiologyBacterial','EtiologyParasitic','EtiologyFungal','EtiologyNonInfectious') \
    .filter(F.col('IsAntibiotics')=="N") \
    .filter(F.col('TreatmentSupplyDate') >= "2013-01-01 00:00:00") \
    .filter(F.col('TreatmentSupplyDate') <= "2021-12-31 00:00:00") \
    .drop('KilogramsToTreat', 'count')

In [0]:
# fill in for the specific dataset:
Type = "OtherTreatment"
EventDate = "TreatmentSupplyDate"
Pen = "PenNumber"
df = dfNotAntibiotics

RightColumns = ["FarmIdentification",
                Pen,
                EventDate,
                "EventType"]
OtherColumns = sorted(list(set(df.columns) - set(RightColumns)))

# change df name:
# create column EventDate and Type, put all other columns in metadata column, drop all other columns
dfOtherTreatmentEventsStruct = df \
    .withColumnRenamed(EventDate, "EventDate") \
    .withColumnRenamed(Pen, "Pen") \
    .withColumn("EventType", F.lit(Type)) \
    .withColumn("MetaData", F.struct(*OtherColumns)) \
    .select("FarmIdentification", "Pen", "EventDate", "EventType", "MetaData")

In [0]:
dfOtherTreatmentEventsStruct.write \
    .format("parquet") \
    .mode("overwrite") \
    .save("//dfOtherTreatmentEventsStruct.parquet")

# first version: 14/11/22

In [0]:
dfNoTreatment = dfFlockPresentationFull \
    .drop('DiagnosisDescription','DiagnosisViral','DiagnosisBacterial','DiagnosisParasitic','DiagnosisFungal','DiagnosisNonInfectious',
        'EtiologyDescription','EtiologyViral','EtiologyBacterial','EtiologyParasitic','EtiologyFungal','EtiologyNonInfectious') \
    .filter(F.col('ContactDate') >= "2013-01-01 00:00:00") \
    .filter(F.col('ContactDate') <= "2021-12-31 00:00:00") \
    .filter(F.col('TreatmentSupplyDate').isNull()) \
    .drop('KilogramsToTreat', 'count')

In [0]:
# fill in for the specific dataset:
Type = "NoTreatment"
EventDate = "TreatmentSupplyDate"
Pen = "PenNumber"
df = dfNoTreatment

RightColumns = ["FarmIdentification",
                Pen,
                EventDate,
                "EventType"]
OtherColumns = sorted(list(set(df.columns) - set(RightColumns)))

# change df name:
# create column EventDate and Type, put all other columns in metadata column, drop all other columns
dfNoTreatmentEventsStruct = df \
    .withColumnRenamed(EventDate, "EventDate") \
    .withColumnRenamed(Pen, "Pen") \
    .withColumn("EventType", F.lit(Type)) \
    .withColumn("MetaData", F.struct(*OtherColumns)) \
    .select("FarmIdentification", "Pen", "EventDate", "EventType", "MetaData")

In [0]:
dfNoTreatmentEventsStruct.write \
    .format("parquet") \
    .mode("overwrite") \
    .save("//dfNoTreatmentEventsStruct.parquet")

# first version: 14/11/22
# edits:
# - 22/11/22 corrected eventDate to 'TreatmentSupplyDate'